In [1]:
# Implement Value at RIsk and Conditional Value at Risk using : Historical Method / Parametric Method / Monte Carlo Method


In [5]:
import pandas as pd
import numpy as np
import datetime as dt
from pandas_datareader import data as pdr
import yfinance as yf
from scipy.stats import norm, t

In [6]:
# Import data : 

def getData(stocks, start, end):
    stock_data= yf.download(stocks, start= start, end=end)
    stock_data=stock_data['Close']
    returns = stock_data.pct_change()
    meanReturns = returns.mean()
    covMatrix = returns.cov()
    return returns, meanReturns, covMatrix

In [7]:
# Portfolio Performance : 

def PortfolioPerformance(weights, meanReturns, covMatrix, Time):
    returns = np.sum(meanReturns*weights)*Time
    std = np.sqrt( np.dot(weights.T, np.dot(covMatrix, weights)) ) * np.sqrt(Time)
    return returns, std

In [8]:
stockList = ['CBA', 'BHP', 'TLS', 'NAB', 'WBC', 'STO']
stocks = [stock+'.AX' for stock in stockList]
endDate = dt.datetime.now()
startDate = endDate - dt.timedelta(days = 800)

returns, meanReturns, covMatrix = getData(stocks, start = startDate, end = endDate)
returns = returns.dropna() 

[*********************100%%**********************]  6 of 6 completed


In [9]:
weights = np.random.random(len(returns.columns))
weights /= np.sum(weights)


In [10]:
returns['Portfolio'] = returns.dot(weights)
print(returns)

              BHP.AX    CBA.AX    NAB.AX    STO.AX    TLS.AX    WBC.AX  \
Date                                                                     
2021-11-24  0.004993  0.003718 -0.000702  0.022255  0.000000 -0.000459   
2021-11-25  0.009937 -0.014509 -0.004569 -0.010160  0.002457 -0.007798   
2021-11-26 -0.015277 -0.010024 -0.023658 -0.048387 -0.007353 -0.025428   
2021-11-29  0.014199 -0.010864 -0.016275 -0.015408 -0.012346 -0.007590   
2021-11-30  0.020741 -0.006398  0.003676 -0.001565  0.017500 -0.019120   
...              ...       ...       ...       ...       ...       ...   
2024-01-25  0.014511  0.001395 -0.002510  0.007884  0.000000  0.005942   
2024-01-29 -0.014093  0.009229  0.011010  0.016949  0.000000  0.009705   
2024-01-30  0.003840  0.001122 -0.000311 -0.005128  0.005000 -0.004597   
2024-01-31  0.004676  0.012753  0.014628  0.011598  0.004975  0.015113   
2024-02-01 -0.003385 -0.029184 -0.021472 -0.011465 -0.002475 -0.016956   

            Portfolio  
Date         

In [11]:
def HistoricalVaR(returns, alpha = 5):
    #Read in a pandas dataframe of returns / a pandas series of returns
    #Output the percentile of distribution at that given alpha confidence
    
    if isinstance(returns, pd.Series):
        return np.percentile(returns, alpha)
    
    elif isinstance(returns, pd.DataFrame):
        return returns.aggregate(HistoricalVaR, alpha=5)
    
    else:
        raise TypeError("Expected returns to be dataFrame or series")
        
print(HistoricalVaR(returns, alpha=5))        
print(HistoricalVaR(returns['Portfolio'], alpha=5))

BHP.AX      -0.029674
CBA.AX      -0.019797
NAB.AX      -0.020811
STO.AX      -0.026774
TLS.AX      -0.012987
WBC.AX      -0.020276
Portfolio   -0.016515
dtype: float64
-0.016515303268001463


In [12]:
def HistoricalCVaR(returns, alpha=5):
    #Read in a pandas dataframe of returns / a pandas series of returns
    #Output the CVaR for a dataframe / Series
        
    if isinstance(returns, pd.Series):
        belowVaR = returns <= HistoricalVaR(returns, alpha=alpha)
        return returns[belowVaR].mean()
    
    elif isinstance(returns, pd.DataFrame):
        return returns.aggregate(HistoricalCVaR, alpha=5)
    
    else:
        raise TypeError("Expected returns to be dataFrame or series")
        
print(HistoricalCVaR(returns, alpha=5))
print(HistoricalCVaR(returns['Portfolio'], alpha=5))

BHP.AX      -0.044897
CBA.AX      -0.027357
NAB.AX      -0.029323
STO.AX      -0.040883
TLS.AX      -0.021096
WBC.AX      -0.029080
Portfolio   -0.022010
dtype: float64
-0.022010082903593263


In [22]:
# 100 day

Time = 1

VaR = -HistoricalVaR(returns['Portfolio'], alpha=5)*np.sqrt(Time)
CVaR = -HistoricalCVaR(returns['Portfolio'], alpha=5)*np.sqrt(Time)
pRet, pStd = PortfolioPerformance(weights, meanReturns, covMatrix, Time)

Initial_investment = 100000

print('Expected Portfolio Return:         ', round(Initial_investment*pRet,2))
print('Value at Risk at 95th Confidence Interval:       ', round(Initial_investment*VaR,2))
print('Conditional VaR at 95th Confidence Interval:      ',round(Initial_investment*CVaR,2))

Expected Portfolio Return:          29.73
Value at Risk at 95th Confidence Interval:        1651.53
Conditional VaR at 95th Confidence Interval:       2201.01


In [23]:
print(f'With the selected PTF, over a {Time} day time horizon, we expect to make:',round(Initial_investment*pRet,2))
print(f'With the selected PTF, over a {Time} day time horizon, we know with a 95% certainty that we are not going to loose more than:', round(Initial_investment*VaR,2)) 
print('Historicaly with this PTF, in the lowest 5th percentile, the expected shortfall that we have is:',round(Initial_investment*CVaR,2)) 
# Historicaly with this PTF, in the lowest 5th percentile, the expected shortfall that we have is: 

With the selected PTF, over a 1 day time horizon, we expect to make: 29.73
With the selected PTF, over a 1 day time horizon, we know with a 95% certainty that we are not going to loose more than: 1651.53
Historicaly with this PTF, in the lowest 5th percentile, the expected shortfall that we have is: 2201.01


In [15]:
#CVaR is mandatory as its giving us the worst case scenario 

In [ ]:
#Parametric VaR and CVaR with Normal distribution and t-distribution

In [ ]:
#The t-distribution, like the normal distribution, is bell-shaped and symmetric, but it has heavier tails, which means that it tends to produce values that fall far from its mean.
#Because the t-distribution has fatter tails than a normal distribution, it can be used as a model for financial returns that exhibit excess kurtosis, which will allow for a more realistic calculation of Value at Risk (VaR) in such cases.

In [24]:
def var_parametric(ptf_return, ptf_std, distribution = 'normal', alpha=5, dof=6):
    # we want to calculate the PTF VaR given a distribution, with known parameters 
    if distribution == 'normal':
        VaR = norm.ppf(1-alpha /100)* ptf_std - ptf_return
    elif distribution == 't-distribution':
        nu = dof
        VaR = np.sqrt((nu-2)/nu)*t.ppf(1-alpha/100, nu)*ptf_std - ptf_return
    else:
        raise TypeError ("Expected distribution to be normal or t-distribution")
    return VaR

def cvar_parametric(ptf_return, ptf_std, distribution = 'normal', alpha=5, dof=6):
    # we want to calculate the PTF CVaR given a distribution, with known parameters 
    if distribution == 'normal':
        CVaR = (alpha /100)**-1 * norm.pdf(norm.ppf(alpha/100))*ptf_std - ptf_return
    elif distribution == 't-distribution':
        nu = dof
        x_anu = t.ppf(alpha/100, nu)
        CVaR = -1/(alpha/100) * (1-nu)**-1 * (nu-2 + x_anu**2) * t.pdf(x_anu, nu)*ptf_std - ptf_return
    else:
        raise TypeError ("Expected distribution to be normal or t-distribution")
    return CVaR

In [25]:
norm_VaR = var_parametric(pRet, pStd)
norm_CVaR = cvar_parametric(pRet, pStd)    

In [26]:
tVar = var_parametric(pRet, pStd, distribution = 't-distribution')
tCVar = cvar_parametric(pRet, pStd, distribution = 't-distribution')

In [28]:
print('Normal Value at Risk at 95th Confidence Interval:       ', round(Initial_investment*norm_VaR,2))
print('Normal Conditional VaR at 95th Confidence Interval:      ',round(Initial_investment*norm_CVaR,2))

print('t-dist Value at Risk at 95th Confidence Interval:       ', round(Initial_investment*tVar,2))
print('t-dist Conditional VaR at 95th Confidence Interval:      ',round(Initial_investment*tCVar,2))

Normal Value at Risk at 95th Confidence Interval:        1492.74
Normal Conditional VaR at 95th Confidence Interval:       1879.51
t-dist Value at Risk at 95th Confidence Interval:        1438.83
t-dist Conditional VaR at 95th Confidence Interval:       1966.01
